In [15]:
import sys
import pandas as pd
import xarray as xr

In [16]:
cd /g/data/xv83/users/at2708/bias_adjustment_acs_qme/qme_dev_hourly/dataset_finder

/g/data/xv83/users/at2708/bias_adjustment_acs_qme/qme_dev_hourly/dataset_finder


In [17]:
# sys.path.insert(0, "/g/data/xv83/users/at2708/bias_adjustment_acs_qme/qme_dev_hourly")
from dataset_finder import *

In [18]:
# if "dataset_finder" in sys.modules:
#     del sys.modules["dataset_finder"]

In [19]:
# import dataset_finder
# print(dataset_finder.__file__)
# from dataset_finder import *


In [20]:
chosen_var = "wbgt"
chosen_ref = "BARRAR2"

In [22]:
%%time
# all_dd_data = get_datasets("ACS_DD", ver = "v1-r1").includes(var = "psl").select(var = "psl")
# all_dd_data = get_datasets("ACS_DD", ver = "v1-r1").includes(var = "sfcWind", exact_match = True).select(var = "sfcWind", exact_match = True)
# all_dd_data = get_datasets("ACS_DD_1h_test", gcm = "ACCESS-CM2", scenario = ("ssp126","ssp370"), timescale = "1hr", var = chosen_var, exact_match = True)
all_dd_data = get_datasets("ACS_DD_1hr_wbgt", gcm = "CMCC-ESM2", scenario = ("historical", "ssp126", "ssp370"), timescale = "1hr", var = chosen_var, exact_match = True)
all_dd_data

CPU times: user 29.4 ms, sys: 20.4 ms, total: 49.9 ms
Wall time: 82.1 ms


,org,gcm,scenario,mdl_run,rcm,ver,timescale,,var,grid,year
0,BOM,CMCC-ESM2,historical,r1i1p1f1,BARPA-R,v1-r1,1hr,,wbgt,AUST-11i,1960 to 2014
1,BOM,CMCC-ESM2,ssp126,r1i1p1f1,BARPA-R,v1-r1,1hr,,wbgt,AUST-11i,2015 to 2100
2,BOM,CMCC-ESM2,ssp370,r1i1p1f1,BARPA-R,v1-r1,1hr,,wbgt,AUST-11i,2015 to 2100


In [23]:
all_dd_data_ref = get_datasets("ACS_DD_1hr_wbgt", gcm = "ERA5", scenario = ("historical"), timescale = "1hr", var = chosen_var, exact_match = True)
all_dd_data_ref

,org,gcm,scenario,mdl_run,rcm,ver,timescale,,var,grid,year
0,BOM,ERA5,historical,hres,BARRAR2,v1,1hr,,wbgt,AUST-11,1979 to 2024


In [24]:
# file_list=all_dd_data.get_files()
# file_list

In [25]:
# file_list_ref=all_dd_data_ref.get_files()
# file_list_ref

In [26]:
%%time
qme_data = get_datasets("ACS_BC", scenario = "ssp", bc = "QME", timescale = "1hr", ref = chosen_ref, var = chosen_var, exact_match = True)
qme_data

CPU times: user 25.8 ms, sys: 3.03 ms, total: 28.9 ms
Wall time: 34.6 ms


In [27]:
runs_to_do = all_dd_data.find_missing(qme_data)
runs_to_do

,org,gcm,scenario,mdl_run,rcm,ver,timescale,,var,grid,year
0,BOM,CMCC-ESM2,historical,r1i1p1f1,BARPA-R,v1-r1,1hr,,wbgt,AUST-11i,1960 to 2014
1,BOM,CMCC-ESM2,ssp126,r1i1p1f1,BARPA-R,v1-r1,1hr,,wbgt,AUST-11i,2015 to 2100
2,BOM,CMCC-ESM2,ssp370,r1i1p1f1,BARPA-R,v1-r1,1hr,,wbgt,AUST-11i,2015 to 2100


In [28]:
keys = ("gcm", "scenario", "mdl_run", "org", "rcm")

with open("/g/data/xv83/users/at2708/bias_adjustment_acs_qme/qme_dev_hourly/jobs_template.sh", 'r') as file:
    template = file.read()

with open("/g/data/xv83/users/at2708/bias_adjustment_acs_qme/qme_dev_hourly/jobs_to_do.sh", 'w') as file:
    file.write(template)
    for dataset in runs_to_do:
        for var in dataset.get_info()["var"]:
            file.write('submit_jobs "{}" "{}" "{}" "{}" "{}" "{}" "{}"\n'.format(*[dataset.data[key] for key in keys], chosen_ref, var))